<a href="https://colab.research.google.com/github/NamrathaKaranth/30-Days-of-Deep-Learning/blob/main/PCB_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Getting the dataset

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"thisissteve","key":"73df294b169ab380c9333b89787d0567"}'}

In [ ]:
 ! mkdir ~/.kaggle
 ! cp kaggle.json ~/.kaggle/
 ! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d namrathakaranth/pcbexperiment

 99% 845M/854M [00:13<00:00, 62.3MB/s]
100% 854M/854M [00:13<00:00, 65.3MB/s]


In [ ]:
!unzip pcbexperiment.zip

Archive:  pcbexperiment.zip
  inflating: dataset/tobeaugmented/defected/20200707_085703.jpg  
  inflating: dataset/tobeaugmented/defected/20200707_090641.jpg  
  inflating: dataset/tobeaugmented/defected/20200707_091300.jpg  
  inflating: dataset/tobeaugmented/defected/20200707_092157.jpg  
  inflating: dataset/tobeaugmented/defected/20200707_092201.jpg  
  inflating: dataset/tobeaugmented/defected/20200707_092731.jpg  
  inflating: dataset/tobeaugmented/defected/20200707_093001.jpg  
  inflating: dataset/tobeaugmented/defected/20200707_101023.jpg  
  inflating: dataset/tobeaugmented/defected/20200707_101034.jpg  
  inflating: dataset/tobeaugmented/defected/20200707_101039.jpg  
  inflating: dataset/tobeaugmented/defected/20200707_101612.jpg  
  inflating: dataset/tobeaugmented/defected/20200707_103238.jpg  
  inflating: dataset/tobeaugmented/defected/20200707_103242.jpg  
  inflating: dataset/tobeaugmented/defected/20200707_104608.jpg  
  inflating: dataset/tobeaugmented/defected/2020

### Data preprocesing 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm

DATADIR = "/content/dataset/tobeaugmented"

CATEGORIES = ["defected", "non-defect"]

training_data = []

def create_training_data():
    for category in CATEGORIES: 

        path = os.path.join(DATADIR,category)  
        class_num = CATEGORIES.index(category)  

        for img in tqdm(os.listdir(path)):  
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE) 
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  
                training_data.append([new_array, class_num])  
            except Exception as e: 
                pass

create_training_data()

print(len(training_data))

100%|██████████| 285/285 [00:24<00:00, 11.83it/s]

0


In [ ]:
import random

random.shuffle(training_data)

In [ ]:
X = []
y = []
IMG_SIZE = 250

for features,label in training_data:
    X.append(features)
    y.append(label)


X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

### Model 

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

import pickle

X = X/255.0

model = Sequential()

model.add(Conv2D(512, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  

model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(np.array(X), np.array(y), batch_size=32, epochs=3, validation_split=0.2)